In [31]:
import csv
import pandas as pd
from mlxtend.frequent_patterns import association_rules

In [32]:
def procedureApriori(T, minSupport):
    candidateItemset = {}
    # frequentItemset is used to keep a running list of all of the frequently occuring sets of data in the code
    frequentItemset = []

    # Generate K1 sets
    for transaction in T:
        for item in transaction:
            if item in candidateItemset:
                candidateItemset[item] += 1
            else:
                candidateItemset[item] = 1
    # Reference used: https://stackoverflow.com/questions/58324419/how-do-i-remove-all-items-in-a-dictionary-with-a-value-less-than-a-variable
    for key in list(candidateItemset):
        if candidateItemset[key] < minSupport:
            del candidateItemset[key]
    frequentItemset = list(candidateItemset)
    if len(candidateItemset) == 0:
        return list(candidateItemset)
    candidateItemset = {}
    countCandidates = {}
    # frequentItemset.sort()

    # Generate K2 Sets
    # Reference used: https://stackoverflow.com/questions/63970483/how-to-check-if-an-array-contains-all-the-elements-of-another-array-if-not-out
    # Candidate Generation
    for i in range(0, len(frequentItemset)):
        for j in range(i + 1, len(frequentItemset)):
            k2Set = []
            if frequentItemset[i] < frequentItemset[j]:
                currentKey = frequentItemset[i] + " " + frequentItemset[j]
                k2Set.append(frequentItemset[i])
                k2Set.append(frequentItemset[j])
            else:
                currentKey = frequentItemset[i] + " " + frequentItemset[j]
                k2Set.append(frequentItemset[j])
                k2Set.append(frequentItemset[i])
            candidateItemset[currentKey] = k2Set
            # Transaction Iteration
            for transaction in T:
                if (
                    len(
                        [
                            h
                            for h in candidateItemset[currentKey]
                            if h not in transaction
                        ]
                    )
                    == 0
                ):
                    if currentKey in countCandidates:
                        countCandidates[currentKey] += 1
                    else:
                        countCandidates[currentKey] = 1
    # Candidate Pruning
    for key in list(candidateItemset):
        if key in countCandidates:
            if countCandidates[key] < minSupport:
                del candidateItemset[key]
                del countCandidates[key]
            else:
                frequentItemset.append(key)
        else:
            del candidateItemset[key]

    if len(candidateItemset) == 0:
        return frequentItemset
    k = 3

    while True:
        countCandidates = {}
        newCandidates = {}
        currentSet = list(candidateItemset)
        # Candidate Generation
        for i in range(0, len(candidateItemset)):
            for h in range(i + 1, len(candidateItemset)):
                match = True
                for x in range(0, len(currentSet[i]) - 2):
                    if len(currentSet[i]) != len(currentSet[h]):
                        continue
                    if currentSet[i][x] != currentSet[h][x]:
                        match = False
                if match == True:
                    newElement = (
                        currentSet[i] + " " + currentSet[h][len(currentSet[h]) - 1]
                    )
                    if newElement not in newCandidates:
                        L = candidateItemset[currentSet[i]]
                        L.append(currentSet[h][len(currentSet[h]) - 1])
                        newCandidates[newElement] = L

        # Transaction Iteration
        newCountCandidates = {}
        for key in newCandidates:
            for transaction in T:
                if len([h for h in newCandidates[key] if h not in transaction]) == 0:
                    if key in newCountCandidates:
                        newCountCandidates[key] += 1
                    else:
                        newCountCandidates[key] = 1

        # Candidate Pruning
        for key in list(newCandidates):
            if key in newCountCandidates:
                if newCountCandidates[key] < minSupport:
                    del newCandidates[key]
                    del newCountCandidates[key]
                else:
                    frequentItemset.append(key)
            else:
                del newCandidates[key]

        # If no candidates after pruning, return
        if len(newCandidates) == 0:
            return frequentItemset
            print("Frequent Sets: " + str(frequentItemset))
            return candidateItemset
        candidateItemset = newCandidates

        k += 1

In [38]:
setSup = 1
inputList = []
with open("./bread.csv", newline="", encoding="utf8") as inputFile:
    reader = csv.reader(inputFile, delimiter=",")
    for row in reader:
        rowList = []
        for i in range(1, len(row)):
            if str(row[i]) != "":
                rowList.append(str(row[i]).decode('utf-16'))
        inputList.append(rowList)

AttributeError: 'str' object has no attribute 'decode'

In [37]:
inputList

[[b'\xff\xfe\xf0\x00x\x01\xdc\x02\xad\x00\xf0\x00x\x01\xdc\x02\xad\x00\xf0\x00x\x01\xdc\x02\xad\x00',
  b'\xff\xfe\xf0\x00x\x01\xa4\x00\xa1\x00\xf0\x00x\x01\xa4\x00\xa1\x00',
  b'\xff\xfe\xf0\x00x\x01\x18 \xac ']]

In [34]:
x = procedureApriori(inputList, setSup)

TypeError: can't concat str to bytes

In [ ]:
frozen_new_x = []
new_x = []
for i in x:
    frozen_new_x.append(frozenset([k for k in i.split(" ")]))
    new_x.append([k for k in i.split(" ")])
frequent_itemsets = pd.DataFrame()
frequent_itemsets["itemsets"] = frozen_new_x
frequent_itemsets["support"] = 1
r = association_rules(frequent_itemsets, support_only=True)

In [ ]:
rules = []
for l in range(len(r["antecedents"].values)):
    rules.append([list(r["antecedents"][l])[0], list(r["consequents"][l])[0]])

In [ ]:
new_rules = []

for i in new_x:
    for rule in rules:
        before = rule[0]
        after = rule[1]
        before_idx = None
        after_idx = None
        for item in i:
            if item == before:
                if before_idx == None:
                    before_idx = i.index(item)
            elif item == after:
                if after_idx == None:
                    after_idx = i.index(item)
        if before_idx != None and after_idx != None:
            if before_idx < after_idx:
                new_rules.append(rule)
print(new_rules)

[['ðŸ˜\xadðŸ˜\xadðŸ˜\xad', 'ðŸ¤¡ðŸ¤¡'], ['ðŸ˜\xadðŸ˜\xadðŸ˜\xad', 'ðŸ‘€'], ['ðŸ¤¡ðŸ¤¡', 'ðŸ‘€']]
